In [7]:
from sqlalchemy import create_engine

#my_db = create_engine(mysql_db_url)
my_db = create_engine('sqlite:///frank_foo.db')

table_name = 'Frank_test0914'
sql_cmd_str = "CREATE TABLE IF NOT EXISTS %s(dtime varchar(20) PRIMARY KEY, temp float)" %(table_name)
resultProxy = my_db.execute(sql_cmd_str)
resultProxy.__dict__

{'context': <sqlalchemy.dialects.sqlite.base.SQLiteExecutionContext at 0x21fb89a2850>,
 'dialect': <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite at 0x21fb89c73a0>,
 'cursor': None,
 'cursor_strategy': <sqlalchemy.engine.cursor.NoCursorDMLFetchStrategy at 0x21fb89530e0>,
 'connection': <sqlalchemy.engine.base.Connection at 0x21fb89a2fa0>,
 '_echo': False,
 '_metadata': <sqlalchemy.engine.cursor._LegacyNoResultMetaData at 0x21fb8907f10>,
 'rowcount': -1,
 '_soft_closed': True}

In [8]:
#建立資料
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(10):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    print(str_dt, temp)
    

2022-08-01 08:00:00 37.0
2022-08-02 08:00:00 37.5
2022-08-03 08:00:00 37.5
2022-08-04 08:00:00 36.9
2022-08-05 08:00:00 36.8
2022-08-06 08:00:00 35.0
2022-08-07 08:00:00 36.0
2022-08-08 08:00:00 37.1
2022-08-09 08:00:00 36.0
2022-08-10 08:00:00 37.0


In [9]:
#插入
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(10):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    resultProxy=my_db.execute(sql_cmd_str)


In [10]:
# 故意插入重複的
dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(5):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    resultProxy=my_db.execute(sql_cmd_str)

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: Frank_test0914.dtime
[SQL: insert into Frank_test0914 (dtime, temp) values('2022-08-01 08:00:00', '36.0')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [11]:
# select
sql_cmd_str = "select * from %s ACS" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-01 08:00:00', 35.7), ('2022-08-02 08:00:00', 37.3), ('2022-08-03 08:00:00', 36.1), ('2022-08-04 08:00:00', 36.1), ('2022-08-05 08:00:00', 36.5), ('2022-08-06 08:00:00', 37.5), ('2022-08-07 08:00:00', 35.6), ('2022-08-08 08:00:00', 36.0), ('2022-08-09 08:00:00', 35.1), ('2022-08-10 08:00:00', 36.0)]


In [12]:
# 做upsert操作 修改、取代 與MySQL 有差異
import datetime
import random

dt1 = datetime.datetime.strptime('2022-08-01 08:00:00','%Y-%m-%d %H:%M:%S') #把時間字串轉為datetime物件
for i in range(15):
    str_dt = (dt1+datetime.timedelta(days=i)).strftime('%Y-%m-%d %H:%M:%S')
    temp = random.randint(350, 375)/10
    #print(str_dt, temp)
    sql_cmd_str = "insert or replace into %s (dtime, temp) values('%s', '%s')" %(table_name, str_dt, temp)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE temp='%s'" %(temp)) mysql 寫法
    resultProxy=my_db.execute(sql_cmd_str)


In [13]:
# select
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-01 08:00:00', 35.4), ('2022-08-02 08:00:00', 35.4), ('2022-08-03 08:00:00', 36.1), ('2022-08-04 08:00:00', 36.4), ('2022-08-05 08:00:00', 35.7), ('2022-08-06 08:00:00', 35.8), ('2022-08-07 08:00:00', 36.4), ('2022-08-08 08:00:00', 37.0), ('2022-08-09 08:00:00', 35.9), ('2022-08-10 08:00:00', 36.9), ('2022-08-11 08:00:00', 35.1), ('2022-08-12 08:00:00', 36.8), ('2022-08-13 08:00:00', 37.3), ('2022-08-14 08:00:00', 36.9), ('2022-08-15 08:00:00', 37.5)]


In [14]:
# select 有條件
sql_cmd_str = "select * from %s where temp>36" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('2022-08-03 08:00:00', 36.1), ('2022-08-04 08:00:00', 36.4), ('2022-08-07 08:00:00', 36.4), ('2022-08-08 08:00:00', 37.0), ('2022-08-10 08:00:00', 36.9), ('2022-08-12 08:00:00', 36.8), ('2022-08-13 08:00:00', 37.3), ('2022-08-14 08:00:00', 36.9), ('2022-08-15 08:00:00', 37.5)]


In [15]:
#Delete
resultProxy=my_db.execute("delete from %s where dtime<'2022-08-05' " %(table_name) )

In [16]:
# select
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('2022-08-05 08:00:00', 35.7), ('2022-08-06 08:00:00', 35.8), ('2022-08-07 08:00:00', 36.4), ('2022-08-08 08:00:00', 37.0), ('2022-08-09 08:00:00', 35.9), ('2022-08-10 08:00:00', 36.9), ('2022-08-11 08:00:00', 35.1), ('2022-08-12 08:00:00', 36.8), ('2022-08-13 08:00:00', 37.3), ('2022-08-14 08:00:00', 36.9), ('2022-08-15 08:00:00', 37.5)]


In [4]:
#練習題：把高雄空氣品質資料寫入資料庫中
import csv

with open('2018_kh_data.csv', 'r') as f:
    cr = csv.reader(f)
    data_list = list(cr)
data_list[0:5]

[['Name',
  'Date',
  'TSP',
  'PM10',
  'Chloride',
  'Nitrate',
  'Sulfate',
  'Lead',
  'Dust'],
 ['大社工業區', '2018年1月', '93', '', '', '', '', '0.029', '3.6'],
 ['大寮區公所', '2018年1月', '79', '', '', '', '', '0.042', '1.54'],
 ['大樹監測站', '2018年1月', '61', '', '', '', '', '0.02', '1.18'],
 ['仁武國中', '2018年1月', '79', '', '', '', '', '0.058', '2.68']]

In [1]:
from sqlalchemy import create_engine

my_db = create_engine('sqlite:///frank_foo.db')

In [2]:
# 我們把tsp, pm10, lead都記下來
table_name = 'Frank_khtsp_0903' #local 端資料表
sql_cmd_str = "CREATE TABLE IF NOT EXISTS %s(sid varchar(20), dtime varchar(20), tsp float, pm10 float, lead float,dust float, PRIMARY KEY(sid, dtime))" %(table_name)
resultProxy = my_db.execute(sql_cmd_str)
resultProxy.__dict__

{'context': <sqlalchemy.dialects.sqlite.base.SQLiteExecutionContext at 0x1a025c10f40>,
 'dialect': <sqlalchemy.dialects.sqlite.pysqlite.SQLiteDialect_pysqlite at 0x1a0272249d0>,
 'cursor': None,
 'cursor_strategy': <sqlalchemy.engine.cursor.NoCursorDMLFetchStrategy at 0x1a027214060>,
 'connection': <sqlalchemy.engine.base.Connection at 0x1a02724ec70>,
 '_echo': False,
 '_metadata': <sqlalchemy.engine.cursor._LegacyNoResultMetaData at 0x1a0271d10a0>,
 'rowcount': -1,
 '_soft_closed': True}

In [5]:
import datetime
import random

for item in data_list[1:]:
    sid = item[0]#['Name']
    dtime = item[1]#['Date']
    tsp = item[2]#['TSP']
    pm10 = item[3]#['PM10']
    lead = item[7]#['Lead']
    dust = item[-1] #dust
    sql_cmd_str = "insert or replace into %s (sid, dtime, tsp, pm10, lead, dust) values('%s', '%s', '%s', '%s', '%s','%s')" %(
        table_name, sid, dtime, tsp, pm10, lead, dust)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE tsp='%s', pm10='%s', lead='%s'" %(tsp, pm10, lead))
    resultProxy=my_db.execute(sql_cmd_str)
    print(sql_cmd_str)

insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('大社工業區', '2018年1月', '93', '', '0.029','3.6')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('大寮區公所', '2018年1月', '79', '', '0.042','1.54')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('大樹監測站', '2018年1月', '61', '', '0.02','1.18')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('仁武國中', '2018年1月', '79', '', '0.058','2.68')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('半屏山電信局', '2018年1月', '104', '67', '0.014','1.21')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('左營國小', '2018年1月', '98', '64', '0.006','1.13')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('永安工業區', '2018年1月', '83', '', '0.052','2.96')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('西子灣中山大學', '

insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('林園工業區', '2018年4月', '78', '', '0.033','3.18')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('河濱國小', '2018年4月', '59', '40', '0.054','1.79')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('阿蓮區公所', '2018年4月', '63', '', '0.037','1.38')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('前鎮國小', '2018年4月', '82', '48', '0.067','4.01')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('莊敬國小', '2018年4月', '67', '50', '0.069','2.01')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('愛國國小', '2018年4月', '78', '41', '0.034','2.61')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('楠梓區清潔隊', '2018年4月', '74', '37', '0.037','2.64')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('路竹地政事務

insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('莊敬國小', '2018年7月', '29', '20', '0.021','1.38')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('愛國國小', '2018年7月', '38', '19', '0.014','1.51')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('楠梓區清潔隊', '2018年7月', '52', '22', '0.014','0.88')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('路竹地政事務所', '2018年7月', '43', '', '0.013','1.79')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('鼓山圖書分館', '2018年7月', '37', '20', '0.022','1.47')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('衛生局中正辦公室', '2018年7月', '47', '19', '0.005','1.35')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('環保局環境檢驗科', '2018年7月', '37', '18', '0.04','0.88')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) va

insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('環保局環境檢驗科', '2018年10月', '104', '51', '0.071','1.97')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('大社工業區', '2018年11月', '103', '', '0.029','2.78')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('大寮區公所', '2018年11月', '98', '', '0.016','1.3')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('仁武國中', '2018年11月', '115', '', '0.1','2.88')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('半屏山電信局', '2018年11月', '104', '53', '0.036','3.35')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('左營國小', '2018年11月', '65', '38', '0.035','1.79')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values('永安工業區', '2018年11月', '93', '', '0.04','2.58')
insert or replace into Frank_khtsp_0903 (sid, dtime, tsp, pm10, lead, dust) values

In [6]:
# 找出某個站的資料: where
sid = '左營國小'
sql_cmd_str = "select * from %s where sid='%s'" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('左營國小', '2018年10月', 89.0, 51.0, 0.024, 2.15), ('左營國小', '2018年11月', 65.0, 38.0, 0.035, 1.79), ('左營國小', '2018年12月', 90.0, 52.0, 0.019, 2.18), ('左營國小', '2018年1月', 98.0, 64.0, 0.006, 1.13), ('左營國小', '2018年2月', 59.0, 32.0, 0.017, 1.5), ('左營國小', '2018年3月', 54.0, 36.0, 0.068, 1.55), ('左營國小', '2018年4月', 61.0, 33.0, 0.028, 1.48), ('左營國小', '2018年5月', 49.0, 38.0, 0.047, 4.69), ('左營國小', '2018年6月', 22.0, 16.0, 0.034, 1.47), ('左營國小', '2018年7月', 37.0, 16.0, 0.022, 1.23), ('左營國小', '2018年8月', 22.0, 14.0, 0.004, 1.23), ('左營國小', '2018年9月', 32.0, 22.0, 0.02, 1.99)]


In [7]:
# 找出某個站的資料, 並排序: where, order
sid = '仁武國中'
sql_cmd_str = "select * from %s where sid='%s' order by dtime DESC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('仁武國中', '2018年9月', 46.0, '', 0.006, 3.38), ('仁武國中', '2018年8月', 19.0, '', -0.009, 4.52), ('仁武國中', '2018年7月', 56.0, '', 0.02, 3.45), ('仁武國中', '2018年6月', 48.0, '', 0.034, 3.25), ('仁武國中', '2018年5月', 44.0, '', 0.058, 1.83), ('仁武國中', '2018年4月', 78.0, '', 0.035, 3.15), ('仁武國中', '2018年3月', 108.0, '', 0.052, 2.63), ('仁武國中', '2018年2月', 86.0, '', 0.033, 3.6), ('仁武國中', '2018年1月', 79.0, '', 0.058, 2.68), ('仁武國中', '2018年12月', 134.0, '', 0.018, 2.67), ('仁武國中', '2018年11月', 115.0, '', 0.1, 2.88), ('仁武國中', '2018年10月', 91.0, '', 0.042, 1.89)]


In [8]:
# 找出某個站的資料, 並排序: where, order
sid = '左營國小'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)


-- data --
[('左營國小', '2018年10月', 89.0, 51.0, 0.024, 2.15), ('左營國小', '2018年11月', 65.0, 38.0, 0.035, 1.79), ('左營國小', '2018年12月', 90.0, 52.0, 0.019, 2.18), ('左營國小', '2018年1月', 98.0, 64.0, 0.006, 1.13), ('左營國小', '2018年2月', 59.0, 32.0, 0.017, 1.5), ('左營國小', '2018年3月', 54.0, 36.0, 0.068, 1.55), ('左營國小', '2018年4月', 61.0, 33.0, 0.028, 1.48), ('左營國小', '2018年5月', 49.0, 38.0, 0.047, 4.69), ('左營國小', '2018年6月', 22.0, 16.0, 0.034, 1.47), ('左營國小', '2018年7月', 37.0, 16.0, 0.022, 1.23), ('左營國小', '2018年8月', 22.0, 14.0, 0.004, 1.23), ('左營國小', '2018年9月', 32.0, 22.0, 0.02, 1.99)]


In [9]:
#排序出了問題
# 先把資料都刪掉: DELETE FROM table_name
sql_cmd_str = "DELETE FROM %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)


In [10]:
# 確認資料
sql_cmd_str = "select * from %s" %(table_name)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
data

[]

In [12]:
# 做upsert操作: 試著改時間格式
import datetime
import random

for item in data_list[1:]:
    sid = item[0]#['Name']
    dtime = item[1]#['Date']
    dtime = dtime.replace('月', '')
    dtime_tmp = dtime.split('年')
    dtime = '%s-%02d' %(dtime_tmp[0], int(dtime_tmp[1]))
    tsp = item[2]#['TSP']
    if tsp=='':
        tsp = 'NULL'
    pm10 = item[3]#['PM10']
    if pm10=='':
        pm10 = 'NULL'
    lead = item[7]#['Lead']
    if lead=='':
        lead = 'NULL'
    dust = item[-1] #dust
    sql_cmd_str = "insert or replace into %s (sid, dtime, tsp, pm10, lead,dust) values('%s','%s', '%s', %s, %s, %s)" %(
        table_name, sid, dtime, tsp, pm10, lead,dust)
    #sql_cmd_str = sql_cmd_str + (" ON DUPLICATE KEY UPDATE tsp=%s, pm10=%s, lead=%s" %(tsp, pm10, lead))
    resultProxy=my_db.execute(sql_cmd_str)
    #print(sql_cmd_str)

In [13]:
# 這樣的時間格式未來如何轉換
dt1 = datetime.datetime.strptime('2018-03','%Y-%m') #把時間字串轉為datetime物件
dt1

datetime.datetime(2018, 3, 1, 0, 0)

In [14]:
#再來取資料，並做排序
# 找出某個站的資料, 並排序: where, order
sid = '左營國小'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('左營國小', '2018-01', 98.0, 64.0, 0.006, 1.13), ('左營國小', '2018-02', 59.0, 32.0, 0.017, 1.5), ('左營國小', '2018-03', 54.0, 36.0, 0.068, 1.55), ('左營國小', '2018-04', 61.0, 33.0, 0.028, 1.48), ('左營國小', '2018-05', 49.0, 38.0, 0.047, 4.69), ('左營國小', '2018-06', 22.0, 16.0, 0.034, 1.47), ('左營國小', '2018-07', 37.0, 16.0, 0.022, 1.23), ('左營國小', '2018-08', 22.0, 14.0, 0.004, 1.23), ('左營國小', '2018-09', 32.0, 22.0, 0.02, 1.99), ('左營國小', '2018-10', 89.0, 51.0, 0.024, 2.15), ('左營國小', '2018-11', 65.0, 38.0, 0.035, 1.79), ('左營國小', '2018-12', 90.0, 52.0, 0.019, 2.18)]


In [15]:
# 找出某個站的資料, 並排序: where, order
sid = '大寮區公所'
sql_cmd_str = "select * from %s where sid='%s' order by dtime ASC" %(table_name, sid)
resultProxy=my_db.execute(sql_cmd_str)
data = resultProxy.fetchall()
print('-- data --')
print(data)

-- data --
[('大寮區公所', '2018-01', 79.0, None, 0.042, 1.54), ('大寮區公所', '2018-02', 91.0, None, 0.018, 2.01), ('大寮區公所', '2018-03', 92.0, None, 0.046, 3.76), ('大寮區公所', '2018-04', 75.0, None, 0.062, 2.28), ('大寮區公所', '2018-05', 43.0, None, 0.021, 2.93), ('大寮區公所', '2018-06', 41.0, None, 0.006, 3.84), ('大寮區公所', '2018-07', 44.0, None, 0.013, 0.65), ('大寮區公所', '2018-08', 14.0, None, 0.01, 1.22), ('大寮區公所', '2018-09', 31.0, None, 0.043, 2.31), ('大寮區公所', '2018-10', 79.0, None, 0.041, 2.13), ('大寮區公所', '2018-11', 98.0, None, 0.016, 1.3), ('大寮區公所', '2018-12', 124.0, None, 0.012, 2.36)]
